In [11]:
from lohrasb.best_estimator import BaseModel
import xgboost
from optuna.pruners import HyperbandPruner
from optuna.samplers._tpe.sampler import TPESampler
from sklearn.model_selection import KFold,train_test_split
import pandas as pd
import numpy as np
import optuna
from sklearn.pipeline import Pipeline
from feature_engine.imputation import (
    CategoricalImputer,
    MeanMedianImputer
    )
from category_encoders import OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score)
from sklearn.metrics import f1_score, mean_absolute_error,r2_score
from sklearn.linear_model import *
from sklearn.svm import *
from xgboost import *
from sklearn.linear_model import *
from catboost import *
from lightgbm import *
from sklearn.neural_network import *
from imblearn.ensemble import *
from sklearn.ensemble import *


# Example 1 :Computer Hardware Data Set (a regression problem)
  
https://archive.ics.uci.edu/ml/datasets/Computer+Hardware

In [12]:
urldata= "https://archive.ics.uci.edu/ml/machine-learning-databases/cpu-performance/machine.data"
# column names
col_names=[
    "vendor name",
    "Model Name",
    "MYCT",
    "MMIN",
    "MMAX",
    "CACH",
    "CHMIN",
    "CHMAX",
    "PRP"
]
# read data
data = pd.read_csv(urldata,header=None,names=col_names,sep=',')
data

vendor name  Model Name  MYCT   MMIN  MMAX  CACH  CHMIN  CHMAX  PRP
adviser          32/60         125   256   6000   256    16    128    198  199
amdahl          470v/7          29  8000  32000    32     8     32    269  253
amdahl         470v/7a          29  8000  32000    32     8     32    220  253
amdahl         470v/7b          29  8000  32000    32     8     32    172  253
amdahl         470v/7c          29  8000  16000    32     8     16    132  132
...                ...         ...   ...    ...   ...   ...    ...    ...  ...
sperry            80/8         124  1000   8000     0     1      8     42   37
sperry   90/80-model-3          98  1000   8000    32     2      8     46   50
sratus              32         125  2000   8000     0     2     14     52   41
wang            vs-100         480   512   8000    32     0      0     67   47
wang             vs-90         480  1000   4000     0     0      0     45   25

[209 rows x 9 columns]

# Train test split

In [13]:
X = data.loc[:, data.columns != "PRP"]
y = data.loc[:, data.columns == "PRP"]
y = y.values.ravel()


X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.33, random_state=42)


# Find feature types for later use

In [14]:
int_cols =  X_train.select_dtypes(include=['int']).columns.tolist()
float_cols =  X_train.select_dtypes(include=['float']).columns.tolist()
cat_cols =  X_train.select_dtypes(include=['object']).columns.tolist()


#  Define estimator and set its arguments  

In [15]:
estimator = LinearRegression()
estimator_params= {
        "fit_intercept": [True, False],
    }


In [16]:
obj = BaseModel.bestmodel_factory.using_optuna(
            estimator=estimator,
            estimator_params=estimator_params,
            measure_of_accuracy="r2_score",
            add_extra_args_for_measure_of_accuracy = False,
            verbose=3,
            n_jobs=-1,
            random_state=42,
            # optuna params
            # optuna study init params
            study=optuna.create_study(
                storage=None,
                sampler=TPESampler(),
                pruner=HyperbandPruner(),
                study_name=None,
                direction="maximize",
                load_if_exists=False,
                directions=None,
            ),
            # optuna optimization params
            study_optimize_objective=None,
            study_optimize_objective_n_trials=10,
            study_optimize_objective_timeout=600,
            study_optimize_n_jobs=-1,
            study_optimize_catch=(),
            study_optimize_callbacks=None,
            study_optimize_gc_after_trial=False,
            study_optimize_show_progress_bar=False,
        )

[I 2022-09-30 14:43:08,222] A new study created in memory with name: no-name-89509fe2-c9fe-466f-a4c0-41004f9661a2


# Build sklearn pipeline

In [17]:


pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # regression model 
            ('obj', obj),


 ])



# Run Pipeline

In [18]:
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)


/Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/optuna/study/study.py:393: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  warnings.warn(
[I 2022-09-30 14:43:08,343] Trial 2 finished with value: 0.9495445619978359 and parameters: {'fit_intercept': 0}. Best is trial 2 with value: 0.9495445619978359.
[I 2022-09-30 14:43:08,346] Trial 0 finished with value: 0.9495445619978359 and parameters: {'fit_intercept': 0}. Best is trial 2 with value: 0.9495445619978359.
[I 2022-09-30 14:43:08,348] Trial 1 finished with value: 0.9495445619978359 and parameters: {'fit_intercept': 0}. Best is trial 2 with value: 0.9495445619978359.
[I 2022-09-30 14:43:08,353] Trial 3 finished with value: 0.9561648243794818 and parameters: {'fit_intercept': 1}. Best is trial 3 with value: 0.9561648243794818.
[I 2022-09-30 14:43:08,359] Trial 4 finished with value: 0.9495445619978359 

Initializing Optuna
sample_weight=None Not a float
sample_weight=None Not a int
multioutput='uniform_average' Not a float
multioutput='uniform_average' Not a int
force_finite=True Not a float
force_finite=True Not a int
sample_weight=None Not a float
sample_weight=None Not a int
multioutput='uniform_average' Not a float
multioutput='uniform_average' Not a int
force_finite=True Not a float
force_finite=True Not a int
sample_weight=None Not a float
sample_weight=None Not a int
multioutput='uniform_average' Not a float
multioutput='uniform_average' Not a int
force_finite=True Not a float
force_finite=True Not a int
{'fit_intercept': 0}
{'fit_intercept': 0}
{'fit_intercept': 0}
r2_score
r2_score
r2_score
{'fit_intercept': 1}
r2_score
{'fit_intercept': 0}
r2_score
{'fit_intercept': 1}
r2_score
{'fit_intercept': 1}
r2_score
{'fit_intercept': 1}
r2_score
{'fit_intercept': 0}
r2_score
{'fit_intercept': 0}
r2_score
LinearRegression(fit_intercept=1)
Initializing Optuna
sample_weight=None Not a f

# Check performance of the pipeline

In [19]:
print('r2 score : ')
print(r2_score(y_test,y_pred))


r2 score : 
0.9402415944774102
